In [2]:
from dotenv import load_dotenv
import os 
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv('env', override=True)
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT=os.getenv('END_POINT')
MODEL_NAME=os.getenv('MODEL_NAME')
print(AZURE_OPENAI_API_KEY[:10])
print(MODEL_NAME)

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true' #true, false
os.environ['LANGCHAIN_PROJECT'] = 'LANG'

if os.getenv('LANGCHAIN_TRACING_V2') == "true":
    print('랭스미스로 추적 중입니다 :', os.getenv('LANGSMITH_API_KEY')[:10])

43b13g4OZS
gpt-4.1-mini
랭스미스로 추적 중입니다 : lsv2_pt_24


## DataLoader

데이터 로더는 원본 자료를 읽어 Document(텍스트+메타데이터)로 바꿔주는 역할

자료의 포맷과 형식, 원리에 따라 다양한 로더가 존재

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("pdf/인터넷서비스_이용약관.pdf", mode="page") #<-- mode="page"로 하면 페이지 단위로 나눠서 로드. 하나의 페이지가 하나의 document가 됨
docs = loader.load()
print(f"문서 개수: {len(docs)}")
print('---------------------------')
print(docs[1].page_content[:500]) #<-- 첫 페이지의 앞 500자 출력
print('---------------------------')
print(docs[1].metadata) #<-- 메타데이터

문서 개수: 232
---------------------------
-  2 - 
제1장  총   칙 
 
제 1조 ( 약관의 적용 ) 
① 주식회사 케이티(이하 "케이티"라 합니다)의 인터넷서비스(이하 "서비스"라 합니다)를 
이용할 때는 이 약관과 전기통신서비스이용기본약관(이하 “기본약관”이라 합니다)을 
함께 적용합니다. 
② 케이티인터넷서비스 이용약관은 지정된 홈페이지(www.kt.com)에 게시하여 공지합니다. 
 
제 2조 ( 정의 ) 
기본약관에서 정의한 용어 이외에 이 약관에서 사용하는 용어의 정의는 다음과 같습니다. 
1. 
KT internet: 케이티가 제공하는 초고속인터넷서비스 
2. 
KT biz kornet: 케이티가 구축한 인터넷망의 이름 또는 인터넷 전용회선서비스 
3. 
KT WiFi: 노트북, PDA, TABLET 등의 단말을 이용하여 가정, 기업, KT WiFi ZONE 
등에서 무선으로 초고속인터넷을 이용할 수 있는 서비스 
4. 
지니 TV VOD: KT internet 회선종단에 지니 TV 단말을 설치하고, 가
---------------------------
{'producer': 'Microsoft® Word Microsoft 365용\udcc0\udc80', 'creator': 'Microsoft® Word Microsoft 365용\udcc0\udc80', 'creationdate': '2025-09-03T15:58:56+09:00', 'source': 'pdf/인터넷서비스_이용약관.pdf', 'file_path': 'pdf/인터넷서비스_이용약관.pdf', 'total_pages': 232, 'format': 'PDF 1.7', 'title': '', 'author': '박현수(상품시너지팀)\udcc0\udc80', 'subject': '', 'keywords': '', 'moddate': '2025-09-03T15:58:56+09:00', 'trapped': '', 'modDate': "D:20250903155

metadata에는 원본 문서의 다양한 정보가 포함되어 있다. 

대표적으로 파일의 이름, page 등이 있다. 메타 정보는 자동으로 뽑히기도 하지만 수동으로 추가할 수도 있다.

In [4]:
for doc in docs:
    doc.metadata['inspector'] = '김진태'
    doc.metadata['page'] = doc.metadata['page'] + 1
    for del_key in ['producer', 'creator', 'modDate']:
        doc.metadata.pop(del_key, None)

### Splitter

읽어들인 데이터는 페이지 단위이므로 너무 많은 정보를 담고 있다.

이를 적절한 의미 단위로 나누는 과정이 필요하다.  

기본적으로 아래 예시처럼 chunk단위로 자르지만 이럴경우 문장이 잘려나갈 수 있으므로 overlap을 두어 안정성을 높인다.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", "  ", " "]
)
splits = splitter.split_documents(docs)
for s in splits[1:5]:
    print(s.page_content)
    print(s.metadata)
    print('-----------------')

-  2 - 
제1장  총   칙 
 
제 1조 ( 약관의 적용 ) 
① 주식회사 케이티(이하 "케이티"라 합니다)의 인터넷서비스(이하 "서비스"라 합니다)를 
이용할 때는 이 약관과 전기통신서비스이용기본약관(이하 “기본약관”이라 합니다)을 
함께 적용합니다. 
② 케이티인터넷서비스 이용약관은 지정된 홈페이지(www.kt.com)에 게시하여 공지합니다. 
 
제 2조 ( 정의 ) 
기본약관에서 정의한 용어 이외에 이 약관에서 사용하는 용어의 정의는 다음과 같습니다. 
1.
{'creationdate': '2025-09-03T15:58:56+09:00', 'source': 'pdf/인터넷서비스_이용약관.pdf', 'file_path': 'pdf/인터넷서비스_이용약관.pdf', 'total_pages': 232, 'format': 'PDF 1.7', 'title': '', 'author': '박현수(상품시너지팀)\udcc0\udc80', 'subject': '', 'keywords': '', 'moddate': '2025-09-03T15:58:56+09:00', 'trapped': '', 'creationDate': "D:20250903155856+09'00'", 'page': 2, 'inspector': '김진태'}
-----------------
1. 
KT internet: 케이티가 제공하는 초고속인터넷서비스 
2. 
KT biz kornet: 케이티가 구축한 인터넷망의 이름 또는 인터넷 전용회선서비스 
3. 
KT WiFi: 노트북, PDA, TABLET 등의 단말을 이용하여 가정, 기업, KT WiFi ZONE 
등에서 무선으로 초고속인터넷을 이용할 수 있는 서비스 
4. 
지니 TV VOD: KT internet 회선종단에 지니 TV 단말을 설치하고, 가정내 정보단말 
및 각종기기를 유무선으로 연결하여 고품질 VOD 및 홈네트워킹 등 다양한
{'creationdate': '2025-09-03T15:58:56+09:00', 'source': 'pdf/인터

### Vectorization

잘려진 데이터는 의미를 간직한 채로 vector로 변경된다. 키워드 중심의 BM25가 있고 아래 예시는 의미 중심의 딥러닝 임베딩 모델을 사용했다.

벡터화 할 때마다 비용과 시간이 들어가므로 변경한 자료는 DB에 저장하는 것이 효율적이다.

In [6]:
import gc
import time
import os
# 기존 변수 정리
if 'vectordb' in globals():
    del vectordb
if 'splits' in globals():
    del splits
if 'docs' in globals():
    del docs
    
gc.collect()
print("✓ Memory cleared")

✓ Memory cleared


In [7]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("1. Loading documents...")
loader = PyMuPDFLoader("pdf/인터넷서비스_이용약관.pdf", mode="page")
docs = loader.load()
print(f"   ✓ Loaded {len(docs)} pages")

print("\n2. Cleaning metadata...")
for doc in docs:
    doc.metadata = {
        'source': doc.metadata.get('source', 'unknown'),
        'page': doc.metadata.get('page', 0) + 1,
        'inspector': '김진태'
    }
print("   ✓ Metadata cleaned")

print("\n3. Splitting documents...")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=80,
    separators=["\n\n", "\n", "  ", " "]
)
splits = splitter.split_documents(docs)
print(f"   ✓ Created {len(splits)} chunks")

# docs 삭제하여 메모리 확보
del docs
gc.collect()
print("   ✓ Memory optimized")

1. Loading documents...
   ✓ Loaded 232 pages

2. Cleaning metadata...
   ✓ Metadata cleaned

3. Splitting documents...
   ✓ Created 447 chunks
   ✓ Memory optimized


In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

print("Creating embeddings model...")
emb = AzureOpenAIEmbeddings(
    model=EMB_MODEL_NAME,
    api_key=AZURE_OPENAI_EMB_API_KEY,
    azure_endpoint=EMB_END_POINT,
    api_version="2024-08-01-preview"
)
print("✓ Embedding model created successfully!")

Creating embeddings model...
✓ Embedding model created successfully!


: 

In [ ]:
from langchain_community.vectorstores import FAISS

print("Creating FAISS index in batches...\n")

BATCH_SIZE = 20
vectordb = None
total = len(splits)
total_batches = (total - 1) // BATCH_SIZE + 1

print(f"Total chunks: {total}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Total batches: {total_batches}\n")

for i in range(0, total, BATCH_SIZE):
    batch = splits[i:i + BATCH_SIZE]
    batch_num = i // BATCH_SIZE + 1
    
    print(f"Batch {batch_num}/{total_batches} ({len(batch)} chunks)...", end=" ", flush=True)
    
    try:
        if vectordb is None:
            vectordb = FAISS.from_documents(batch, emb)
            print("✓ (initialized)")
        else:
            vectordb.add_documents(batch)
            print("✓")
        
        gc.collect()
        time.sleep(1)
        
    except Exception as e:
        print(f"\n✗ Error: {e}")
        import traceback
        traceback.print_exc()
        raise

print("\n✅ FAISS index created successfully!")

# 즉시 저장!
print("\nSaving FAISS index...")
try:
    vectordb.save_local("faiss_index_jupyter")
    print("✓ Saved to 'faiss_index_jupyter'")
except Exception as e:
    print(f"✗ Save failed: {e}")

# splits 삭제하여 메모리 확보
print("\nCleaning memory...")
if 'splits' in globals():
    del splits
gc.collect()
print("✓ Memory cleaned")

# 간단한 테스트 (메모리 부담 적게)
print("\nQuick test...")
try:
    test_docs = vectordb.similarity_search("테스트", k=1)
    print(f"✓ Search works! Found: {test_docs[0].page_content[:80]}...")
except Exception as e:
    print(f"⚠ Search test skipped (this is OK, index is saved)")

print("\n✅ Complete! FAISS index is ready to use.")

Creating FAISS index in batches...

Total chunks: 447
Batch size: 20
Total batches: 23

Batch 1/23 (20 chunks)... ✓ (initialized)
Batch 2/23 (20 chunks)... ✓
Batch 3/23 (20 chunks)... ✓
Batch 4/23 (20 chunks)... ✓
Batch 5/23 (20 chunks)... ✓
Batch 6/23 (20 chunks)... ✓
Batch 7/23 (20 chunks)... ✓
Batch 8/23 (20 chunks)... ✓
Batch 9/23 (20 chunks)... ✓
Batch 10/23 (20 chunks)... ✓
Batch 11/23 (20 chunks)... ✓
Batch 12/23 (20 chunks)... ✓
Batch 13/23 (20 chunks)... ✓
Batch 14/23 (20 chunks)... ✓
Batch 15/23 (20 chunks)... ✓
Batch 16/23 (20 chunks)... ✓
Batch 17/23 (20 chunks)... ✓
Batch 18/23 (20 chunks)... ✓
Batch 19/23 (20 chunks)... ✓
Batch 20/23 (20 chunks)... ✓
Batch 21/23 (20 chunks)... ✓
Batch 22/23 (20 chunks)... ✓
Batch 23/23 (7 chunks)... ✓

✅ FAISS index created successfully!

Saving FAISS index...
✓ Saved to 'faiss_index_jupyter'

Cleaning memory...
✓ Memory cleaned

Quick test...


In [ ]:
print("Testing search functionality...\n")

try:
    # 간단한 검색
    test_docs = vectordb.similarity_search("테스트", k=3)
    print(f"✓ Found {len(test_docs)} documents\n")
    
    for i, doc in enumerate(test_docs, 1):
        print(f"--- Result {i} ---")
        print(f"Content: {doc.page_content[:150]}...")
        print(f"Metadata: {doc.metadata}")
        print()
    
    print("✅ Search test passed!")
    
except Exception as e:
    print(f"✗ Search failed: {e}")
    import traceback
    traceback.print_exc()

In [9]:
# #로컬에 저장된 db를 불러올 때
# vectordb = FAISS.load_local("faiss_index", emb, allow_dangerous_deserialization=True)

### Retriever

벡터 DB는 보통 리트리버를 포함하고 있다.

리트리버는 관련있는 정보를 검색하여 return한다.

**kargs**
- search_type : 관련 문서를 고르는 알고리즘
    - similarity : 유사한 문서 추출
    - similarity_score_threshold : 임계값 이상인 것들만 추출해서 관련 없는 정보를 배제
    - mmr : 너무 유사한 것들만 있으면 중복이 심하다. 다양한 정보를 가져와 편향을 방지

- k : 상위 k개 최종 선별



In [11]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})
# retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 10})
# retriever = vectordb.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 10, "score_threshold": 0.5})

In [9]:
emb_text = emb.embed_query("안녕하세요")
print(emb_text[:5])
print(len(emb_text)) #text-embedding-3-small 모델은 어떤 문장이 들어가도 1536차원 벡터를 리턴

[-0.002531265141442418, -0.06127675995230675, -0.008443817496299744, 0.031540773808956146, 0.031089577823877335]
1536


In [12]:
for doc in retriever.get_relevant_documents("가정의 달에 가입하면 혜택이 뭐야?"):  
    print(doc.page_content)
    print(doc.metadata['source'], doc.metadata['page'], doc.metadata['author'])
    # 유사도 출력
    print('--------------------------------')

/var/folders/jr/r6q5x5h966554q8401g1rk2m0000gn/T/ipykernel_31389/825489379.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  for doc in retriever.get_relevant_documents("가정의 달에 가입하면 혜택이 뭐야?"):


: 

만약 청크의 유사도 점수도 보고 싶다면 'retriever'대신 'vectordb'로 검색

In [10]:
for doc, score in vectordb.similarity_search_with_score("가정의 달에 가입하면 혜택이 뭐야?"):  
    print(doc.page_content)
    print(doc.metadata['source'], doc.metadata['page'], doc.metadata['author'])
    print(score)
    print('--------------------------------')

: 

(심화) 만약에 찾아온 정보가 너무 많거나 혹은 정보의 밀도가 낮은 경우 context를 바로 프롬프트에 넣기 전에 요약을 할 수 있다.      
물론 컨텍스트를 압축하는데 비용이 들어가므로 수행하지 않아도 된다.

In [ ]:
# 컨텍스트를 압축하는 코드
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

compressor = LLMChainExtractor.from_llm(llm)
compressed_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever # compressor를 설정하면 압축된 context를 가져옴
)

## RAG 만들기

기본적으로 일반챗봇과 동일하지만 context를 넣는 위치가 존재하고 프롬프트에 context를 강하게 참조라하고 명령

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 통신사 고객상담을 위한 이용약관 문서 기반 QA 어시스턴트다. 아래 규칙을 지켜라.\n"
     "1) 제공된 컨텍스트(표/본문)에서만 근거를 찾아 답한다.\n"
     "2) 컨택스트 내에 해당하는 정보가 없으면 모른다고 말한다. 추측 금지.\n"
     "3) 숫자/조건/예외는 정확히 인용하라.\n"
     "4) 표 내용이라면 행/열 헤더를 함께 언급해 맥락을 명확히 한다.\n"
     "5) 맨 마지막에 레퍼런스를 추가한다.\n"),
    ("human", 
     "질문: {question}\n\n"
     "컨텍스트:\n{context}\n\n"
     "한국어로 간결하게 답해.")
])

체인을 만들어 실행해봅시다.

RunnablePassthrough는 입력 변수를 바로 통과시키는 역할을 합니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

parser = StrOutputParser()

def list2string(docs):
    # 원문을 이어붙임
    return "\n\n-----\n\n".join(d.page_content for d in docs)

# RAG 체인: (질문) → DB에서 검색 → 컨텍스트 추출 → 프롬프트에 추가 → LLM → 출력 텍스트
rag_chain = (
    {"question": RunnablePassthrough(),   # 입력 쿼리는 question과 retriever로 나뉘어 들어간다. 
     "context": retriever | list2string} #<-- 러너블람다를 사용하면 안정성을 높일 수 있다.
    | prompt
    | llm
    | parser
)


주어진 쿼리를 토대로 가장 관련된 내용을 찾아 답변하는 것을 확인할 수 있습니다

In [ ]:
query = "오피스IP팩은 어떤 서비스야?"
print(rag_chain.invoke(query))

: 

### 출처를 추가하기

지금 컨택스트에 원본 문서의 정보를 넣음으로써 사실을 기반으로 한 서비스를 만들 수 있게 되었다. 

그러나 근거가 된 컨택스트가 어디에 있는지 알 수 없어 사실 검증이 어렵다. 원본 데이터의 출처를 같이 출력하여 신뢰감을 높일 수 있다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 통신사 고객상담을 위한 이용약관 문서 기반 QA 어시스턴트다. 아래 규칙을 지켜라.\n"
     "1) 제공된 컨텍스트(표/본문)에서만 근거를 찾아 답한다.\n"
     "2) 컨택스트 내에 해당하는 정보가 없으면 모른다고 말한다. 추측 금지.\n"
     "3) 숫자/조건/예외는 정확히 인용하라.\n"
     "4) 표 내용이라면 행/열 헤더를 함께 언급해 맥락을 명확히 한다.\n"
     "5) 마지막에 출처 파일과 페이지를 함께 제시한다.\n"),
    ("human", 
     "질문: {question}\n\n"
     "컨텍스트:\n{context}\n\n"
     "답변: ")
])

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

parser = StrOutputParser()

def list2string(docs):
    # 원문 뒤에 메타 정보를 추가
    return "\n\n-----\n\n".join(d.page_content + f"\n\n <출처>: {d.metadata['source']} - {d.metadata['page']}" for d in docs)

rag_chain = (
    {"question": RunnablePassthrough(),   
     "context": retriever | list2string}
    | prompt
    | llm
    | parser
)

print(rag_chain.invoke("오피스IP팩은 어떤 서비스야?"))

오피스IP팩(오피스IP 서비스)은 다수의 IP를 필요로 하는 소규모 오피스 대상 서비스입니다. 이 서비스는 개인, 개인사업자, 법인사업자를 대상으로 제공되며, KT WiFi AP 장비를 함께 제공하는 경우도 있습니다. 오피스IP 서비스는 유동IP와 고정IP 형태로 제공되며, 유동IP는 인터넷서비스와 함께 제공되고, 고정IP는 유동IP 기반으로 회선 당 최대 3개까지 제공됩니다. 또한, 이용고객의 사용 단말 수를 제한하지 않습니다.

특히, 오피스IP넷 플러스(GiGA WiFi premium) 서비스는 KT WiFi AP 장비를 함께 제공하며, 유동IP와 고정IP 옵션이 있습니다. 오피스IP 서비스는 계약기간에 따른 요금할인도 제공됩니다.

요약하면, 오피스IP팩은 소규모 오피스에서 다수의 IP를 필요로 할 때 적합한 인터넷 서비스로, KT의 WiFi 장비와 함께 제공되며, 유동IP와 고정IP 옵션을 선택할 수 있고, 단말 수 제한이 없는 서비스입니다.

출처: pdf/인터넷서비스_이용약관.pdf, 페이지 199~203, 210~211


In [ ]:
print((retriever | list2string).invoke("오피스IP팩은 어떤 서비스야?"))

-  200 - 
오피스넷 
다수의 IP를 필요로 하는 소규모 
오피스 대상 서비스 
개인, 개인사업자, 
법인사업자 
종별 
최대속도 
요금 
(IP당) 
오피스넷  
슈퍼프리미엄 
10G 
132,000원 
오피스넷  
프리미엄플러스 
5G 
99,000원 
오피스넷  
프리미엄 
2.5G 
71,500원 
오피스넷  
에센스 
1G 
64,900원 
오피스넷  
베이직 
500M 
56,100원 
오피스넷 슬림 
100M 
49,500원 
오피스넷 고정IP 
슈퍼프리미엄 
10G 
137,500원 
오피스넷 고정IP 
프리미엄플러스 
5G 
104,500원 
오피스넷 고정IP 
프리미엄 
2.5G 
77,000원 
오피스넷 고정IP 
에센스 
1G 
70,400원 
ㅇ오피스넷 기본상품은 
 유동IP로 
인터넷서비스를 함께 
제공함  
 
ㅇ오피스넷 고정IP는 
오피스넷 기본상품 
기반으로 제공되며 
회선 당 최대 3개까지 
제공  
 
· 오피스넷은 
이용고객의 사용단말 
수를 제한하지 않음

 <출처>: pdf/인터넷서비스_이용약관.pdf - 199

-----

-  211 - 
제공하는 보안서비스 
 IP Net은 기본으로 제공되는 고정 IP 외에는 부가서비스로 고정 IP를 추가로 제공받을 수 없습니다. 
 오피스IP 서비스 및 오피스넷(와이파이 및 보안와이파이 포함), “멀티넷”은 해당되지 않습니다.  
 KT SafeNet 서비스는 타임코디, 크린아이, 인터넷가족안심, 가족안심패키지, 안심 플러스, 패키지용 안심 플러스, 인터넷 놀이터, 인터넷  
지킴이 서비스와 동시 가입이 불가합니다 
6. 요금할인 
할인제도 
내 용 
대상요금 및 할인율/할인액 
계약기간 
요금할인 
이용고객이 사용기간을 
사전 계약하는 경우 
계약기간 동안 요금할인 
제공 
종별 
1년약정 
2년약정 
3년약정 
4년약정 
5년약정 
OFFICE 
5% 
10% 
15% 
20% 
- 
오피스IP 서비스 
5,500원 
11,000원 
16,500원 
- 
- 
오피

## RAG 통합코드

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("pdf/인터넷서비스_이용약관.pdf", mode="page") 
docs = loader.load()

print('데이터 로드 완료')

from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=80,
    separators=["\n\n", "\n", "  ", " "]
)
splits = splitter.split_documents(docs)

print('분할 완료')

from langchain_community.vectorstores import Chroma, FAISS
from langchain_openai import AzureOpenAIEmbeddings

emb = AzureOpenAIEmbeddings(
    model=EMB_MODEL_NAME,                      
    api_key=AZURE_OPENAI_EMB_API_KEY,
    azure_endpoint=EMB_END_POINT,
    api_version="2024-08-01-preview"
)
vectordb = FAISS.from_documents(splits,emb)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 10})
print('임베딩 및 저장 완료')

데이터 로드 완료
분할 완료
임베딩 및 저장 완료


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 통신사 고객상담을 위한 이용약관 문서 기반 QA 어시스턴트다. 아래 규칙을 지켜라.\n"
     "1) 제공된 컨텍스트(표/본문)에서만 근거를 찾아 답한다.\n"
     "2) 컨택스트 내에 해당하는 정보가 없으면 모른다고 말한다. 추측 금지.\n"
     "3) 숫자/조건/예외는 정확히 인용하라.\n"
     "4) 표 내용이라면 행/열 헤더를 함께 언급해 맥락을 명확히 한다.\n"
     "5) 마지막에 출처 파일과 페이지를 함께 제시한다.\n"),
    ("human", 
     "질문: {question}\n\n"
     "컨텍스트:\n{context}\n\n"
     "답변: ")
])

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

parser = StrOutputParser()

def list2string(docs):
    # 원문 뒤에 메타 정보를 추가
    return "\n\n-----\n\n".join(d.page_content + f"\n\n <출처>: {d.metadata['source']} - {d.metadata['page']}" for d in docs)

rag_chain = (
    {"question": RunnablePassthrough(),   
     "context": retriever | list2string}
    | prompt
    | llm
    | parser
)

print(rag_chain.invoke("KT 사장님 배달POS의 비용은?"))

KT 사장님 배달POS의 비용은 월 22,000원입니다.  
또한, 서비스 가입 시 최초 1회 2개월 간 월정액 요금 전액 할인이 적용됩니다.  
출시 기념 프로모션으로 기간 내 서비스 가입 시 월정액 요금 할인 혜택이 1개월 연장되며, 기간은 2022.9.01부터 2023.2.28까지입니다.  
추가 단말 이용대수는 1대까지로 하며, 추가 단말 이용 요금은 월 5,500원입니다.  
또한, 으랏차차 패키지 결합상품에 가입된 인터넷 회선에 사장님 배달POS를 가입하고 할인 신청 시 사장님 배달POS 요금을 2,200원 할인하여 적용합니다.

출처: pdf/인터넷서비스_이용약관.pdf - 93, 95
